In [2]:
import os
import sys
import json
import re
import argparse
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from Bio.Seq import Seq
from itertools import chain
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [5]:
folder_name = "C_psittaci"

In [6]:
# Function for parsing assembly number
def get_assembly_number(pre_pattern1, string):
    pre_pattern2 = r"_[\d]+?\.gbk"
    first_bound = re.search(pre_pattern1, string)
    second_bound = re.search(pre_pattern2, string)
    result = string[first_bound.end():(second_bound.start())]
    return result

In [7]:
# Uploading the data 
current_path = f"../{folder_name}/data/{folder_name}_annotate/" # Path directory
pre_pattern1 = f"{folder_name}"
gb_records= []  # Reasserting the gb_records list
gbk_files = os.listdir(current_path)

In [8]:
for gbk in tqdm(gbk_files):
    assembly_number = get_assembly_number(pre_pattern1, gbk)
    link = f"{current_path}{gbk}"
    gbk_object = SeqIO.read(link, 'genbank')
    gb_records.append((gbk_object, assembly_number))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:02<00:00, 22.78it/s]


### Creating first table

In [9]:
# Uploading info about which DNA is plasmid
with open(f"../{folder_name}/data//{folder_name}_plasmid_code1.json", "r") as tuples:
    tuples = json.load(tuples)
with open(f"../{folder_name}/data/{folder_name}_plasmid_code2.json", "r") as dna_type:
    dna_type = json.load(dna_type)
tuples = [tuple(x) for x in tuples]
plasmid_code = dict(zip(tuples, dna_type))

In [10]:
# Parsing
locus_tag, start_codone, source, n_sequence, aa_sequence, DNA_source, \
product, cog, p_c_unity, first_n, last_n, strand_n, \
length_n = [], [], [], [], [], [], [], [], [], [], [], [], []
for record_number in tqdm(range(len(gb_records))):
    for feature_number in range(1, len(gb_records[record_number][0].features)):
        if gb_records[record_number][0].features[feature_number].type == "CDS":
            
            ## Column 1 -- uniq id
            if 'locus_tag' in gb_records[record_number][0].features[feature_number].qualifiers.keys():
                locus_tag.append(gb_records[record_number][0].features[feature_number].qualifiers['locus_tag'][0])
            else:
                locus_tag.append("absent")
                
            ## Column 2 -- source
            source.append(gb_records[record_number][0].name)
            
            ## Column 3, 11-14 -- nucleotide sequence & 7 -- start-codone
            first = gb_records[record_number][0].features[feature_number].location.start
            last = gb_records[record_number][0].features[feature_number].location.end
            strand = gb_records[record_number][0].features[feature_number].location.strand
            first_n.append(first)
            last_n.append(last)
            strand_n.append(strand)
            length_n.append(last - first)
            if strand == 1:
                sequence = str(gb_records[record_number][0].seq[first:last])
                n_sequence.append(sequence)
                start_codone.append(sequence[0:3])
            elif strand == -1:
                sequence = str(gb_records[record_number][0].seq.complement()[first:last])[::-1]
                n_sequence.append(sequence)
                start_codone.append(sequence[0:3])
            else:
                n_sequence.append("absent")
                
            ## Column 4 -- aminoacid sequence
            if 'translation' in gb_records[record_number][0].features[feature_number].qualifiers.keys():
                aa_sequence.append(gb_records[record_number][0].features[feature_number].qualifiers['translation'][0])
            else:
                aa_sequence.append("absent")
                
            ## Column 5 -- plasmid or chromosome
            index = re.search(r"[\d]+?_[\d]+$", gb_records[record_number][0].name)
            pc_key = tuple([int(i) for i in index.group().split("_")])
            if plasmid_code[pc_key] == "plasmid":
                DNA_source.append("plasmid")
            else:
                DNA_source.append("chromosome")
            
            # Column 6 -- protein, coding by this gene
            if 'product' in gb_records[record_number][0].features[feature_number].qualifiers.keys():
                product.append(gb_records[record_number][0].features[feature_number].qualifiers['product'][0])
            else:
                product.append("absent")
                
            ## Column 7 -- COG number
            if 'db_xref' in gb_records[record_number][0].features[feature_number].qualifiers.keys():
                cog.append(gb_records[record_number][0].features[feature_number].qualifiers["db_xref"][0][4:])
            else:
                cog.append("absent")
            
            ## Column 8 -- number of molecule (for identification and further plasmid marking)
            p_c_unity.append(gb_records[record_number][1])
            

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:09<00:00,  5.12it/s]


In [11]:
df1 = pd.DataFrame(
    {
        "id": pd.Series(locus_tag),
        "source": pd.Series(source),
        "n_sequence": pd.Series(n_sequence),
        "aa_sequence": pd.Series(aa_sequence),
        "type_of_DNA_source": pd.Series(DNA_source),
        "start_codone": pd.Series(start_codone),
        "product": pd.Series(product),
        "cog": pd.Series(cog),
        "p_c_unity": pd.Series(p_c_unity),
        "first": pd.Series(first_n),
        "last": pd.Series(last_n),
        "strand": pd.Series(strand_n),
        "length": pd.Series(length_n),
    }
)

In [13]:
assembly_stata = pd.DataFrame(df1.groupby("p_c_unity").apply(lambda x: x.query("start_codone == 'ATG'").shape[0]), )
assembly_stata.columns = ["atg_number"]
df1 = df1.merge(assembly_stata, on="p_c_unity")
k = assembly_stata["atg_number"].median()
df1 = df1.query("atg_number > @k/2")
df1

,id,source,n_sequence,aa_sequence,type_of_DNA_source,start_codone,product,cog,p_c_unity,first,last,strand,length,atg_number
0,LEBECCED_00001,C_psittaci4_1,GTGGTAGACTTTGCAACTTTAGGTGGTAGACTTTTGGTTGAAAAAA...,MVDFATLGGRLLVEKNLDCARLVHYKILRLSMSGLSHHHKSRLFLT...,plasmid,GTG,hypothetical protein,absent,4,16,1033,-1,1017,888
1,LEBECCED_00002,C_psittaci4_1,GTGAGTAAATTAACTAAAGAAGCATCCGCTTTTTTTCAAAAGAATC...,MSKLTKEASAFFQKNQENTTKEFLKKEFAMDVFSVSLSDIEKEQIE...,plasmid,GTG,hypothetical protein,absent,4,1161,1902,-1,741,888
2,LEBECCED_00003,C_psittaci4_1,GTGAAAACGCTAGCATTTTGTTCTTTTAAAGGTGGTACTGGGAAAA...,MKTLAFCSFKGGTGKTTLSLNIGSNLAQVSRKKVLLVDLDPQANLT...,plasmid,GTG,Sporulation initiation inhibitor protein Soj,COG1192,4,1898,2678,-1,780,888
3,LEBECCED_00004,C_psittaci4_1,GTGGTTTTATATGAAAAAATCATACGTCTATTTACTGGAGAGATTA...,MVLYEKIIRLFTGEIMNDKQNMTNDFIKIVKDVEKDFPELDIKMKV...,plasmid,GTG,hypothetical protein,absent,4,2708,3062,-1,354,888
4,LEBECCED_00005,C_psittaci4_1,ATGGGTAATTCTGGTTTTTACTTAAATGACACCCAAAATTGTGTAT...,MGNSGFYLNDTQNCVFADNIKLGQMESPLQDQQLILGTKSTPTAAK...,plasmid,ATG,Virulence plasmid protein pGP3-D,absent,4,3111,3906,-1,795,888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27330,OCODBOIF_01040,C_psittaci11_1,ATGCTTACGGATAAAATTATATTATTTGTTACCGAAGACAGTAGTA...,MLTDKIILFVTEDSSISLQLKEFAQNVEYNIVVSSTLTDTPEADLI...,chromosome,ATG,Atypical response regulator protein ChxR,absent,11,1158901,1159615,1,714,846
27331,OCODBOIF_01041,C_psittaci11_1,ATGATACTTGCATCTGTTTTATTTTCCCCCGAAGATTTTTCTTTTC...,MILASVLFSPEDFSFPELITEAYYTWDILALIDRKLSTHVFSGIHG...,chromosome,ATG,Bifunctional protein GlmU,absent,11,1159674,1160289,1,615,846
27332,OCODBOIF_01042,C_psittaci11_1,ATGGTCGTTATGGATATTTTCGAAGCTTACCGCGTAATGATAGAAA...,MVVMDIFEAYRVMIETGIEDSLDEFGPKGFSIRAPVEYALTSGGKR...,chromosome,ATG,Farnesyl diphosphate synthase,absent,11,1160304,1161186,1,882,846
27333,OCODBOIF_01044,C_psittaci11_1,ATGAAAAACCTTATCGATAATAACTTAGTGAGATTTAAAAATATTT...,MKNLIDNNLVRFKNISKTKQGIFVNFKVRGEKGGASFTASIAVDIE...,chromosome,ATG,hypothetical protein,absent,11,1161285,1161540,-1,255,846


For recoding COGs from ids to categories I have downloaded tsvs with all categories and appropriate COGs and have created python dictionary.
Firstly I have uploaded tsvs to folder on google drive.

In [3]:
# Creating cog dictionary
cog_dict = {}
current_path = "../data/bigcog/"
cog_cats = os.listdir(current_path)
for cog_cat in tqdm(cog_cats):
    table = pd.read_csv(os.path.join(current_path, cog_cat), sep="\t")
    if len(table.Cat):
        cat = table.Cat.tolist()
        cat_ful = list(map(lambda x: x.split(" "),cat))
        cat_ful_single = list(chain.from_iterable(cat_ful))
        key = max(cat_ful_single, key = cat_ful_single.count)
        value = table.COG.tolist()
        cog_dict[key] = value
cog_dict["S"].append("absent")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 453.53it/s]


In [15]:
cog

['absent',
 'absent',
 'COG1192',
 'absent',
 'absent',
 'absent',
 'COG0305',
 'absent',
 'COG0113',
 'COG1726',
 'absent',
 'absent',
 'COG1448',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'COG0648',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'COG1137',
 'absent',
 'COG2877',
 'absent',
 'COG0564',
 'absent',
 'absent',
 'absent',
 'COG0187',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'COG3869',
 'absent',
 'COG0233',
 'COG0528',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'COG0396',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'COG1475',
 'COG1123',
 'COG0444',
 'absent',
 'absent',
 'COG0126',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'absent',
 'COG0587',
 'COG2271',
 'absent',
 'COG0124',
 'COG0173',
 'absent',
 'COG0219',
 'COG0526',
 'absent

On the next stage I have created 25 new columns for dataset, each contained data if protein has this COG.

In [19]:
# Adding 25 columns to dataset
for key in tqdm(cog_dict.keys()):   # Passing over COG categories
    cog_list = []             # For each category create list
    for cog_id in cog:        # Passing out every item
        if cog_id not in cog_dict[key]:     # For every gene, if gene's id not in dict
            cog_list.append(0)              # Mark it
        else:                               
            cog_list.append(1)              # Else mark another
    df1[key] = pd.Series(cog_list)   # Creating 25 new columns

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 23.28it/s]


In [27]:
df1.query("id == 'APHAOMEK_00005'")["S"]

3098    1
Name: S, dtype: int64

In [73]:
df1.to_csv(f"../{folder_name}/data/First_table.csv", index=False)

In [74]:
for source in tqdm(set(df1["p_c_unity"])):
    subset = df1[df1["p_c_unity"] == source]
    with open (f"../{folder_name}/data/orto_rows/{folder_name}{str(source)}.fasta", "w") as protein_fasta:
        for index, row in subset.iterrows():
            #if row['type_of_the_gene'] != "pseudogene":   # It will be actual, if we will found pseudogenes again
            protein_fasta.write(">")
            protein_fasta.write(row["id"])
            protein_fasta.write("_")
            protein_fasta.write(row["source"])
            protein_fasta.write("\n")
            protein_fasta.write(row["aa_sequence"])
            protein_fasta.write("\n")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 621.60it/s]

9
26
6
17
12
0
3
16
10
21
7
20
19
13
2
15
1
22
18
23
24
4
14
25
5
11
8
